In [ ]:
] add CSV

In [ ]:
] add DataFrames

In [ ]:
using CSV
using DataFrames

In [ ]:
alldata = CSV.read("data.csv", DataFrame);

In [ ]:
size(alldata)

In [ ]:
data = alldata[5:end, :];

In [ ]:
size(data)

In [ ]:
data.x

In [ ]:
using WGLMakie

In [ ]:
f = Figure()
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize=20, color=:red)
f

In [ ]:
b_eye = 50.
m_eye = 2.

In [ ]:
xx = LinRange(50, 250, 20)

In [ ]:
f = Figure()
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize=20, color=:red)

yy_eye = xx .* m_eye .+ b_eye
lines!(xx, yy_eye)

f

In [ ]:
] add Optim

In [ ]:
using Optim

In [ ]:
function objective(parameters)
    b = parameters[1]
    m = parameters[2]
    @show b, m
    return abs(b - 3.4) + abs(m - 17)
end

In [ ]:
function objective_2(parameters)
    b = parameters[1]
    m = parameters[2]

    y_pred = b .+ m .* data.x
    
    return sum(abs.(data.y .- y_pred))
end

In [ ]:
starting_params = [b_eye + 0., m_eye]
result = optimize(objective_2, starting_params)

In [ ]:
b_abs, m_abs = Optim.minimizer(result)

In [ ]:
f = Figure()
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize=20, color=:red)

yy_eye = xx .* m_eye .+ b_eye
lines!(xx, yy_eye, color=:orange)

yy_abs = xx .* m_abs .+ b_abs
lines!(xx, yy_abs, color=:green)

f

In [ ]:
function objective_gauss(parameters, x, y, sigma)
    b = parameters[1]
    m = parameters[2]

    y_pred = b .+ m .* x
    
    return -sum(
        -log.(sigma * sqrt(2 * π)) .-0.5 .* (y .- y_pred).^2 / sigma.^2)
end

In [ ]:
#function shim(p)
#    return objective_abs(p, data.x, data.y, data.sigma_y)
#end

In [ ]:
starting_params = [b_eye + 0., m_eye]
result = optimize(p -> objective_gauss(p, data.x, data.y, data.sigma_y),
                  starting_params)

In [ ]:
b_gauss, m_gauss = Optim.minimizer(result)

In [ ]:
f = Figure()
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize=20, color=:red)

yy_eye = xx .* m_eye .+ b_eye
lines!(xx, yy_eye, color=:orange)

yy_abs = xx .* m_abs .+ b_abs
lines!(xx, yy_abs, color=:green)

yy_gauss = xx .* m_gauss .+ b_gauss
lines!(xx, yy_gauss, color=:purple, linewidth=3)

f

In [ ]:
@show data.x;

In [ ]:
size(data, 1)

In [ ]:
ndata = size(data,1)

B_jack = zeros(ndata)
M_jack = zeros(ndata)

for i in 1:ndata
    
    xcopy = copy(data.x)
    deleteat!(xcopy, i)

    ycopy = copy(data.y)
    deleteat!(ycopy, i)

    scopy = copy(data.sigma_y)
    deleteat!(scopy, i)

    starting_params = [b_eye + 0., m_eye]
    result = optimize(p -> objective_gauss(p, xcopy, ycopy, scopy),
                      starting_params)
    @assert Optim.converged(result)
    b_jack, m_jack = Optim.minimizer(result)
    
    B_jack[i] = b_jack
    M_jack[i] = m_jack 
end

In [ ]:
@assert 2+2 == 4

In [ ]:
f = Figure()
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize=20, color=:red)

for i in 1:ndata
    yy_jack = xx .* M_jack[i] .+ B_jack[i]
    lines!(xx, yy_jack, color=:green)
end

yy_gauss = xx .* m_gauss .+ b_gauss
lines!(xx, yy_gauss, color=:purple, linewidth=3)

f

In [ ]:
f = Figure()
Axis(f[1, 1])
scatter!(B_jack, M_jack, markersize=20, color=:red)

plot!([b_gauss], [m_gauss], markersize=30)

f

In [ ]:
using Statistics

In [ ]:
mean(B_jack), std(B_jack) .* sqrt((ndata - 1) / ndata)

In [ ]:
mean(M_jack), std(M_jack) .* sqrt((ndata - 1) / ndata)

In [ ]:
f = Figure()
Axis(f[1, 1])
errorbars!(alldata.x, alldata.y, alldata.sigma_y)
scatter!(alldata.x, alldata.y, markersize=20, color=:red)
f

In [ ]:
starting_params = [b_eye + 0., m_eye]
result = optimize(p -> objective_gauss(p, alldata.x, alldata.y, alldata.sigma_y),
                  starting_params)
@assert Optim.converged(result)
b_bad, m_bad = Optim.minimizer(result)

In [ ]:
f = Figure()
Axis(f[1, 1])
errorbars!(alldata.x, alldata.y, alldata.sigma_y)
scatter!(alldata.x, alldata.y, markersize=20, color=:red)

yy_bad = xx .* m_bad .+ b_bad
lines!(xx, yy_bad, color=:purple, linewidth=3)

f

In [ ]:
function objective_outliers(parameters, x, y, sigma)
    b = parameters[1]
    m = parameters[2]

    frac_bad = 0.01
    like_bad = frac_bad * (1. / 600.)

    y_pred = b .+ m .* x
    like_good = (1. - frac_bad) * 1 ./(sqrt(2*π) .* sigma) .* exp.(-0.5 * (y .- y_pred).^2 ./ sigma.^2)
    like = like_bad .+ like_good
    loglike = log.(like)

    return -sum(loglike)
end

In [ ]:
starting_params = [b_eye + 0., m_eye]
result = optimize(p -> objective_outliers(p, alldata.x, alldata.y, alldata.sigma_y),
                  starting_params)
@assert Optim.converged(result)
b_out, m_out = Optim.minimizer(result)

In [ ]:
starting_params = [700 + 0., -0.5]
result = optimize(p -> objective_outliers(p, alldata.x, alldata.y, alldata.sigma_y),
                  starting_params)
@assert Optim.converged(result)
b_punk, m_punk = Optim.minimizer(result)

In [ ]:
f = Figure()
Axis(f[1, 1])
errorbars!(alldata.x, alldata.y, alldata.sigma_y)
scatter!(alldata.x, alldata.y, markersize=20, color=:red)

yy_bad = xx .* m_bad .+ b_bad
lines!(xx, yy_bad, color=:purple, linewidth=3)

yy_out = xx .* m_out .+ b_out
lines!(xx, yy_out, color=:green, linewidth=3)

yy_punk = xx .* m_punk .+ b_punk
lines!(xx, yy_punk, color=:blue, linewidth=3)

f